In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Reading in the data
cal = pd.read_csv('calendar.csv')
stv = pd.read_csv('sales_train_validation.csv')
ss = pd.read_csv('sample_submission.csv')
sellp = pd.read_csv('sell_prices.csv')

In [14]:
cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [21]:
stv.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [7]:
sellp.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [2]:
#Downcast in order to save memory
import numpy as np
import pandas as pd

def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i, t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:  # Corrected this line
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

# Apply downcast function to the dataframes
stv_downcast = downcast(stv)
sellp_downcast = downcast(sellp)
cal_downcast = downcast(cal)


In [3]:
df = pd.melt(stv, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

In [4]:
# Merge with calendar dataframe on column 'd' using left join
df = pd.merge(df, cal, on='d', how='left')

# Merge with sellp dataframe on columns ['store_id','item_id','wm_yr_wk'] using left join
df = pd.merge(df, sellp, on=['store_id','item_id','wm_yr_wk'], how='left')

In [5]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


In [8]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming df is your merged dataset with datetime column 'date' and target variable 'sold'

# Convert 'date' column to datetime if it's not already in datetime format
df['date'] = pd.to_datetime(df['date'])

# Set 'date' column as index
df.set_index('date', inplace=True)

# Train-test split
train_size = int(len(df) * 0.8)  # 80% train, 20% test
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Fit ARIMA model
# Choose the order (p, d, q) based on ACF and PACF plots or using a grid search
p, d, q = 5, 1, 1  # Example order, may need to adjust
model = ARIMA(train['sold'], order=(p, d, q))
arima_model = model.fit()

# Forecast
forecast = arima_model.forecast(steps=len(test))[0]

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test['sold'], forecast))
print("RMSE:", rmse)

# Visualize results if needed
import matplotlib.pyplot as plt

plt.plot(test.index, test['sold'], label='Actual')
plt.plot(test.index, forecast, label='Forecast')
plt.legend()
plt.show()



c:\Users\Catarina Ferreira\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Catarina Ferreira\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Catarina Ferreira\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Catarina Ferreira\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sar

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Preprocessing: Handling NaN values
# Forward fill NaN values in 'sell_price' column
df['sell_price'].fillna(method='ffill', inplace=True)

# Forward fill NaN values in other columns if needed
df.fillna(method='ffill', inplace=True)

# Define function for exponential smoothing prediction
def exp_smooth_predict(series, forecast_steps):
    model = ExponentialSmoothing(series, trend='add').fit()
    forecast = model.forecast(steps=forecast_steps)
    return forecast

# Assuming 'd' represents time series data
time_series = df.groupby('d')['sold'].sum()

# Perform exponential smoothing prediction
forecast = exp_smooth_predict(time_series, forecast_steps=30)

print("Forecasted sales for the next 30 days:")
print(forecast)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# Step 1: Preprocess the dataset to handle missing values
# Let's impute missing values with the mean for numerical columns
imputer = SimpleImputer(strategy='mean')
df[['sell_price']] = imputer.fit_transform(df[['sell_price']])

# Step 2: Prepare the dataset for time series forecasting
# Convert 'd' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Step 3: Split the dataset into training and testing sets
# Assuming the last 30 days are for prediction
train = df[df['date'] < df['date'].max() - pd.Timedelta(days=30)]
test = df[df['date'] >= df['date'].max() - pd.Timedelta(days=30)]

# Step 4: Choose and train a machine learning model
# Random Forest Regressor is chosen for demonstration
features = ['dept_id', 'cat_id', 'store_id', 'state_id', 'sell_price']
target = 'sold'

X_train, X_test = train[features], test[features]
y_train, y_test = train[target], test[target]

model = RandomForestRegressor()
model.fit(X_train, y_train)

# Step 5: Evaluate the model's performance
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

train_rmse = mean_squared_error(y_train, train_pred, squared=False)
test_rmse = mean_squared_error(y_test, test_pred, squared=False)

print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

# Step 6: Use the trained model to make predictions for the next 30 days
# Let's predict for the next 30 days
future_dates = pd.date_range(df['date'].max() + pd.Timedelta(days=1), periods=30, freq='D')
future_data = pd.DataFrame({'date': future_dates, 'dept_id': 'dept_id_value', 'cat_id': 'cat_id_value', 'store_id': 'store_id_value', 'state_id': 'state_id_value', 'sell_price': df['sell_price'].mean()})  # Assuming average sell_price for future dates

future_pred = model.predict(future_data[features])
print("Predictions for the next 30 days:", future_pred)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Preprocessing
# Handle missing values
imputer = SimpleImputer(strategy='mean')
df['sell_price'] = imputer.fit_transform(df[['sell_price']])

# Encode categorical variables
encoder = LabelEncoder()
df['cat_id'] = encoder.fit_transform(df['cat_id'])
df['dept_id'] = encoder.fit_transform(df['dept_id'])
df['store_id'] = encoder.fit_transform(df['store_id'])
df['state_id'] = encoder.fit_transform(df['state_id'])

# Set up Deep Learning Model
# Split data into training and testing sets
X = df.drop(columns=['sold'])
y = df['sold']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model architecture
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
X_train = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

# Make predictions for the next 30 days
last_30_days_data = X[-30:].values.reshape((1, 30, 1))
predictions = model.predict(last_30_days_data)
print(f'Predictions for the next 30 days: {predictions}')